In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_dataset = pd.read_excel('Concrete_Data.xls')  
# last column: Concrete strength is the y
# data shape: (1030, 9)
raw_dataset.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [3]:
raw_dataset.columns = ['cement', 'slag', 'ash', 'water','superplastic','coarseagg','fineagg','age','strength']
raw_dataset.head()

,cement,slag,ash,water,superplastic,coarseagg,fineagg,age,strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [4]:
# check if there's any missing values
pro_dataset = raw_dataset.copy()

# No missing values in the dataset
pro_dataset.isnull().sum()

cement          0
slag            0
ash             0
water           0
superplastic    0
coarseagg       0
fineagg         0
age             0
strength        0
dtype: int64

In [5]:
# Replace the outliers with median
for cols in pro_dataset.columns[:-1]:
    Q1 = pro_dataset[cols].quantile(0.25)
    Q3 = pro_dataset[cols].quantile(0.75)
    iqr = Q3 - Q1

    lower = Q1 - 1.5*iqr
    upper = Q3 + 1.5*iqr
    pro_dataset.loc[(pro_dataset[cols] < lower) | (pro_dataset[cols] > upper), cols] = pro_dataset[cols].median()

In [6]:
#Splitting the data into independent(x) and dependent variables(y)

raw_X = raw_dataset.drop('strength', axis = 1)
raw_y = raw_dataset['strength']
pro_X = pro_dataset.drop('strength', axis = 1)
pro_y = pro_dataset['strength']

# Spit the data
from sklearn.model_selection import train_test_split

# Load your data into X (features) and y (response) arrays

# Split the data into training and test sets, using a fixed number of test samples
num_test_samples = 130
pro_X_train, pro_X_test, pro_y_train, pro_y_test = train_test_split(pro_X, pro_y, test_size=num_test_samples, random_state=42)
raw_X_train, raw_X_test, raw_y_train, raw_y_test = train_test_split(raw_X, raw_y, test_size=num_test_samples, random_state=42)

# Print the shapes of the resulting datasets

# Training data shape: (900, 8) (900,). Test data shape: (130, 8) (130,)
print("Training data shape:", pro_X_train.shape, pro_y_train.shape)
print("Test data shape:", pro_X_test.shape, pro_y_test.shape)



Training data shape: (900, 8) (900,)
Test data shape: (130, 8) (130,)


# MAE as loss function on processed data

In [7]:
def Gd_mae_multi(X, y, num_iterations = 1000, learning_rate = 0.00000001, stopping_threshold=1e-10):
    previous_cost = None
    y = np.array(y).reshape(-1, 1)
   
    new_X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
#     current_m = np.random.rand(9,)
    current_m = np.zeros(9,)
    n = float(len(new_X))
    
    for i in range(num_iterations):
        
        y_pred = np.dot(new_X , current_m)
        
        # Calculating the current cost
        current_cost = mean_absolute_error(y, y_pred)
#         print(f"Iteration {i+1}: Cost {current_cost}")
 
        # If the change in cost is less than or equal to
        # stopping_threshold we stop the gradient descent
        if previous_cost and abs(previous_cost - current_cost) <= stopping_threshold:
            break
        previous_cost = current_cost 
        
        # compute gradients
        dL_dy_pred = (1/n)* sum(np.sign(y_pred - y))

        # compute gradients
        d_m = np.dot(new_X.T, dL_dy_pred)
#         print(d_m.shape)
#         print(f"Gradients: {d_m}")

         
        # Updating weights and bias
        current_m = current_m - (learning_rate * d_m)
                 
#         Printing the parameters for each 1000th iteration
#         print(f"Iteration {i+1}:Weight {current_m}")
        

    return current_m 

In [8]:
# MSE
def mean_squared_error(y_true, y_predicted):
    cost = np.sum((y_true - y_predicted)**2) / len(y_true)
    return cost

In [9]:
# MAE
def mean_absolute_error(y, y_pred):
    diff = y_pred - y
    abs_diff = np.absolute(diff)
    mean_diff = abs_diff.mean()
    return mean_diff

# Multivariate using training/ testing set with MAE as loss function

In [10]:
# Multi variate

m = Gd_mae_multi(pro_X_train, pro_y_train)

pro_X_train_temp = np.concatenate((np.ones((pro_X_train.shape[0], 1)), pro_X_train), axis=1)
y_pred = np.dot(pro_X_train_temp, m)

# R^2
n= len(pro_X_train_temp)

MSE = mean_squared_error(y_pred , pro_y_train)

var = (1/n)*np.sum((pro_y_train - np.mean(pro_y_train))**2)
# R_sq 
r_sq_multiV = 1 - (MSE/var)
print('multivariate model on processed training data:', r_sq_multiV)

multivariate model on processed training data: 0.00323548909485738


# Multivariate using testing set with MAE as loss function

In [11]:
# Multi variate

m = Gd_mae_multi(pro_X_train, pro_y_train)

pro_X_test_temp = np.concatenate((np.ones((pro_X_test.shape[0], 1)), pro_X_test), axis=1)
y_pred = np.dot(pro_X_test_temp , m)
# print(y_pred.shape)
# y_pred = np.dot(pro_X_train, m)

# # R^2
n= len(pro_X_test_temp)

MSE = mean_squared_error(y_pred , pro_y_test)

var = (1/n)*np.sum((pro_y_test - np.mean(pro_y_test))**2)
# R_sq 
r_sq_multiV = 1-(MSE/var)
print('multivariate model on processed testing data:', r_sq_multiV)


multivariate model on processed testing data: 0.0039197923459005635


# Univariate model with training/testing data with MAE as loss function

In [12]:
def Gd_mae_uni(X, y, num_iterations = 1000, learning_rate = 0.0001, stopping_threshold=1e-6):
    previous_cost = None
    current_m = 0
    current_b = 0
    n = float(len(X))
    
    for i in range(num_iterations):
        
        y_pred = np.dot(X , current_m) + current_b
        # Calculating the current cost
        current_cost = mean_absolute_error(y, y_pred)

 
        # If the change in cost is less than or equal to
        # stopping_threshold we stop the gradient descent
        if previous_cost and abs(previous_cost - current_cost) <= stopping_threshold:
            break
        previous_cost = current_cost         
        
       
        # compute MAE loss and its derivative
#         loss = np.mean(np.abs(y - y_pred))
        dL_dy_pred = np.sign(y_pred - y) / n

        # compute gradients
        d_m = np.dot(X.T, dL_dy_pred)
        d_b = np.sum(dL_dy_pred)

        # update parameters
        current_m -= learning_rate * d_m
        current_b -= learning_rate * d_b
#         Printing the parameters for each 1000th iteration
#         print(f"Iteration {i+1}:Weight {current_m} cost {current_cost}")
        

    return current_m, current_b

In [13]:
columns = ['cement', 'slag', 'ash', 'water','superplastic','coarseagg','fineagg','age','strength']


In [14]:
for i in range(8):
    x_1 = np.array(pro_X_train.iloc[:,i])
    x_2 = np.array(pro_X_test.iloc[:,i])
    y_1 = np.array(pro_y_train)
    y_2 = np.array(pro_y_test)
    
    m, b = Gd_mae_uni(x_1, y_1)

    
    y_pred_1 = np.dot(x_1 , m) + b
    y_pred_2 = np.dot(x_2 , m) + b


    # R^2
    n= len(x_1)

    MSE_1 = mean_squared_error(y_pred_1 , y_1)
    MSE_2 = mean_squared_error(y_pred_2 , y_2)
    var_1 = np.var(y_1)
    var_2 = np.var(y_2)
    # R_sq 
    r_sq_uniV_1= 1 - (MSE_1/var_1)
    print(f"processed on MAE {columns[i]}: training r-squared {r_sq_uniV_1}")
    r_sq_uniV_2= 1 - (MSE_2/var_2)
    print(f"processed on MAE{columns[i]}: testing r-squared {r_sq_uniV_2}")
    print("===========")

processed on MAE cement: training r-squared 0.1701955242042018
processed on MAEcement: testing r-squared 0.12421130510087297
processed on MAE slag: training r-squared -2.351449331194098
processed on MAEslag: testing r-squared -2.4548935582263933
processed on MAE ash: training r-squared -2.9303238389029835
processed on MAEash: testing r-squared -2.565926395599996
processed on MAE water: training r-squared -0.2202687223724462
processed on MAEwater: testing r-squared -0.24438730857304836
processed on MAE superplastic: training r-squared -3.581274952001653
processed on MAEsuperplastic: testing r-squared -3.780766430203765
processed on MAE coarseagg: training r-squared -4.571984746775337
processed on MAEcoarseagg: testing r-squared -4.820163513370206
processed on MAE fineagg: training r-squared -2.0144937393192226
processed on MAEfineagg: testing r-squared -2.0966165186010306
processed on MAE age: training r-squared -0.838129309414787
processed on MAEage: testing r-squared -0.93300072725107

# Multivariate model on training/ testing data with Ridge  

In [15]:

# Ridge Regression

class RidgeRegression_multi() :

    def __init__( self, learning_rate, iterations, l2_penality ) :

        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l2_penality = l2_penality

    # Function for model training
    def fit( self, X, Y ) :

        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.M= np.zeros( self.n )

        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning

        for i in range( self.iterations ) :
            self.update_weights()
        return self

    # Helper function to update weights in gradient descent

    def update_weights( self ) :
        Y_pred = self.predict( self.X )

        # calculate gradients
        d_m = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) + ( 2 * self.l2_penality * self.M) ) / self.m
        d_b = - 2 * np.sum( self.Y - Y_pred ) / self.m

        # update weights
        self.M= self.M- self.learning_rate * d_m
        self.b = self.b - self.learning_rate * d_b
        return self

    # Hypothetical function h( x )
    def predict( self, X ) :
        return X.dot( self.M) + self.b




In [16]:
x_1 = np.array(pro_X_train)
x_2 = np.array(pro_X_test)
y_1 = np.array(pro_y_train)
y_2 = np.array(pro_y_test)


model = RidgeRegression_multi( iterations = 10000, learning_rate = 0.0000001, l2_penality = 2 )
model.fit( x_1 , y_1 )
y_pred_1= model.predict(x_1)

y_pred_2= model.predict(x_2)


# R^2
n= len(x_1)

MSE_1 = mean_squared_error(y_pred_1 , y_1)
MSE_2 = mean_squared_error(y_pred_2 , y_2)
var_1 = np.var(y_1)
var_2 = np.var(y_2)
# R_sq 
r_sq_uniV_1= 1 - (MSE_1/var_1)
print(f"processed multi ridge: training r-squared {r_sq_uniV_1}")
r_sq_uniV_2= 1 - (MSE_2/var_2)
print(f"processed multi ridge: testing r-squared {r_sq_uniV_2}")
print("===========")

processed multi ridge: training r-squared 0.6959603866530505
processed multi ridge: testing r-squared 0.678777641199731


# Univariate model on training/testing data with Ridge

In [17]:

# Ridge Regression

class RidgeRegression_uni() :

    def __init__( self, learning_rate, iterations, l2_penality ) :

        self.learning_rate = learning_rate
        self.iterations = iterations
        self.l2_penality = l2_penality

    # Function for model training
    def fit( self, X, Y ) :

        # no_of_training_examples, no_of_features
        self.m, self.n = X.shape

        # weight initialization
        self.W = np.zeros( self.n )

        self.b = 0
        self.X = X
        self.Y = Y

        # gradient descent learning

        for i in range( self.iterations ) :
            self.update_weights()
        return self

    # Helper function to update weights in gradient descent

    def update_weights( self ) :
        Y_pred = self.predict( self.X )

        # calculate gradients
        dW = ( - ( 2 * ( self.X.T ).dot( self.Y - Y_pred ) ) + ( 2 * self.l2_penality * self.W ) ) / self.m
        db = - 2 * np.sum( self.Y - Y_pred ) / self.m

        # update weights
        self.W = self.W - self.learning_rate * dW
        self.b = self.b - self.learning_rate * db
        return self

    # Hypothetical function h( x )
    def predict( self, X ) :
        return X.dot( self.W ) + self.b




In [18]:
for i in range(8):
    x_1 = np.array(pro_X_train.iloc[:,i],).reshape(-1, 1)
    x_2 = np.array(pro_X_test.iloc[:,i],).reshape(-1, 1)
    y_1 = np.array(pro_y_train)
    y_2 = np.array(pro_y_test)
    
    
    model = RidgeRegression_uni( iterations = 10000, learning_rate = 0.000001, l2_penality = 2 )
    model.fit( x_1 , y_1 )
    y_pred_1= model.predict(x_1)
        
    y_pred_2= model.predict(x_2)


    # R^2
    n= len(x_1)

    MSE_1 = mean_squared_error(y_pred_1 , y_1)
    MSE_2 = mean_squared_error(y_pred_2 , y_2)
    var_1 = np.var(y_1)
    var_2 = np.var(y_2)
    # R_sq 
    r_sq_uniV_1= 1 - (MSE_1/var_1)
    print(f"processed {columns[i]}: training r-squared {r_sq_uniV_1}")
    r_sq_uniV_2= 1 - (MSE_2/var_2)
    print(f"processed {columns[i]}: testing r-squared {r_sq_uniV_2}")
    print("===========")

processed cement: training r-squared 0.1737684548124504
processed cement: testing r-squared 0.13782947376472976
processed slag: training r-squared -2.2774492058810307
processed slag: testing r-squared -2.4043297735345637
processed ash: training r-squared -2.8715207733945287
processed ash: testing r-squared -2.5119818090698067
processed water: training r-squared -0.20113730443665712
processed water: testing r-squared -0.23214561852230942
processed superplastic: training r-squared -1.4859187861600942
processed superplastic: testing r-squared -1.498052340531999
processed coarseagg: training r-squared -0.08539308016463432
processed coarseagg: testing r-squared -0.08342589610709128
processed fineagg: training r-squared -0.12923977962559596
processed fineagg: testing r-squared -0.0987678337539144
processed age: training r-squared -0.766640642357032
processed age: testing r-squared -0.8360973290086047
